In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import seaborn as sns
import torch
import pandas as pd
import pyro

In [4]:
import spaceoracle
from spaceoracle import SpaceOracle

In [5]:
import scanpy as sc
ad = sc.read_h5ad('../data/flowsig/adata_train_flowsig.h5ad')
ad

AnnData object with n_obs × n_vars = 10277 × 4998
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_kmeans'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'commot-user_database-info', 'flowsig_network', 'hvg', 'log1p', 'nsf_info', 'rctd_cluster_colors'
    obsm: 'X_flow', 'X_gem', 'X_spatial', 'commot-user_database-sum-receiver', 'commot-user_database-sum-sender', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'commot-user_database-Angptl2-Pirb', 'commot-user_database-Angptl4-Cdh5

In [6]:
from spaceoracle.tools.ligrec_network import LigandReceptorNetwork
ligrec_net = LigandReceptorNetwork(ad)
ligrec_net.gl_dict

defaultdict(list,
            {'Dennd5b': ['C4b'],
             'Il24': ['C4b'],
             'Iglc4': ['C4b'],
             'Igkv12-46': ['C4b'],
             'Map3k9': ['C4b'],
             'Gm26532': ['C4b'],
             'Chst3': ['C4b'],
             'Gm47917': ['C4b'],
             'Chac1': ['C4b'],
             'Msx1': ['C4b'],
             'Clec10a': ['Cxcl12', 'C3'],
             'Cyp2e1': ['Cxcl12', 'C3'],
             'Lum': ['Cxcl12', 'C3'],
             'Ucp1': ['Cxcl12', 'C3'],
             'Dab2': ['Cxcl12', 'C3'],
             'Lyve1': ['Cxcl12', 'C3'],
             'Cma1': ['Cxcl12', 'C3'],
             'Cd5l': ['Cxcl12', 'C3'],
             'Cd59a': ['Cxcl12', 'C3'],
             'Tpsb2': ['Cxcl12', 'C3'],
             'Gm16341': ['Ptn'],
             'Rps13-ps5': ['Ptn'],
             '4930426D05Rik': ['Ptn'],
             'Gapt': ['Ptn'],
             'Tacstd2': ['Ptn'],
             'Gm14302': ['Ptn'],
             'Gm8805': ['Ptn'],
             'Gm16283': ['Ptn']

In [7]:
target_gene = 'Il22ra2'
target_gene in ad.var_names

True

In [55]:
from spaceoracle.models.probabilistic_estimators import ProbabilisticPixelAttentionLR
pyro.clear_param_store()

estimator = ProbabilisticPixelAttentionLR(
    ad, lrn=ligrec_net, target_gene=target_gene, layer='imputed_count')

In [57]:
%%time

# NOTE: runs but not sure if it works lmao

estimator.fit(
    annot='rctd_cluster', 
    max_epochs=10, 
    learning_rate=7e-4, 
    spatial_dim=32,
    batch_size=512,
    alpha=0.1,
    cache=False
)

Fitting 7 models in parallel... with 7/64 cores
🌍️ Generating spatial 32x32 maps...


CPU times: user 1min 18s, sys: 906 ms, total: 1min 19s
Wall time: 1min 17s
